## Bibliography

* https://dev.to/louisbertin/download-all-flickr-photos-of-a-user-with-only-20-lines-of-python-4gh7
* https://medium.com/@jameld.pro/secure-your-api-keys-b123f30ac014

## Import flickr api keys and secrets


In [29]:
import flickr_api
from config import api_key, api_secret

flickr_api.set_keys(api_key = api_key, api_secret = api_secret)


In [30]:
import os

USER_NAME = '15K Nocturna Valencia Banco Mediolanum'
OUTPUT_DIR = 'flickr_images/' + USER_NAME + '/'

# Create folder
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

try:
    user = flickr_api.Person.findByUserName(USER_NAME)
    print('User found: ' + user.username)
except Exception as e:
    print(e, 'User not found')


User found: 15K Nocturna Valencia Banco Mediolanum


In [31]:
from tqdm.notebook import tqdm

pages_nb = user.getPublicPhotos().info.pages
total = user.getPublicPhotos().info.total

print('total photos: ' + str(total))

current = 0
for page_nb in tqdm(range(1, pages_nb+1)):
    for index, photo in tqdm(enumerate(user.getPublicPhotos(page=page_nb))):
        try:
            sizes = photo.getSizes()
            biggest_size = list(sizes.keys())[-1]
            filename = photo.title.replace("/", "-") + "_" + photo.id
            dest_path = OUTPUT_DIR +  filename
            if not os.path.exists(dest_path):
                photo.save(dest_path, size_label = biggest_size)
            current += 1

        except Exception as e:
            print(e, 'Error saving photo: ' + filename)
            continue
        
print('Total photos downloaded: ' + str(current))

total photos: 909


  0%|          | 0/10 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Total photos downloaded: 909
